<a href="https://colab.research.google.com/github/frasercrichton/data-investigation-conspiracy-aotearoa/blob/main/analysis/Pattern_of_Life.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clean and Extract Data


In [85]:
import pandas as pd
import cv2
from deep_translator import GoogleTranslator
videos_df = pd.read_json('../data/source/video-2023-09-14.json', convert_dates=['createTime'])

processed_dir = '../data/processed/'

## Parse the Videos JSON file

- Extract:
  - Comments
  - Likes
- Translate any text

In [12]:
videos_df = videos_df.drop(columns=[
                                    'author',
                                    'challenges',
                                    'collected',
                                    'contents', 
                                    'digged', 
                                    'duetDisplay', 
                                    'forFriend',
                                    'itemCommentStatus', 
                                    'privateItem', 
                                    'secret', 
                                    'shareEnabled', 
                                    'stitchDisplay', 
                                    'officalItem',
                                    'originalItem',
                                    'duetEnabled',
                                    'stitchEnabled',
                                    ])
def translate_text(text):
    translation = GoogleTranslator(source='it', target='en').translate(text)
    return translation  

def parse_desc(desc):
    return translate_text(desc)

def parse_stats(stats):
    return pd.Series([stats['collectCount'], stats['commentCount'], stats['diggCount'], stats['playCount'], stats['shareCount']])

def parse_text_extra(extra_text):
    if extra_text is None:
        return pd.Series([extra_text, extra_text])
    
    extra_text_as_json = pd.json_normalize(extra_text)    
    hashtags = ', '.join(extra_text_as_json['hashtagName'].values)
    return pd.Series([hashtags, translate_text(hashtags)])

def parse_warn_info(warn_info):
    if warn_info is None:
        return warn_info
    
    return warn_info[0]['text']

def parse_video(video):    
    return video['zoomCover']['960']

# desc: translate the video description into English
videos_df['desc_en'] = videos_df.apply(lambda row: parse_desc(row['desc']), axis=1)

# textExtra: extract a list of hashtags
videos_df[['textExtra', 'textExtra_en']] = videos_df.apply(lambda row: parse_text_extra(row['textExtra']), axis=1)

# stats: turn the like, comment, etc. counts into columns
videos_df[['collectCount', 'commentCount', 'diggCount', 'playCount', 'shareCount']] = videos_df.apply(lambda row: parse_stats(row['stats']), axis=1)
videos_df = videos_df.drop(columns=['stats'])

# warnInfo: extract any content warning text 
videos_df['warnInfo'] = videos_df.apply(lambda x: parse_warn_info(x['warnInfo']), axis=1)

# Videos - zoomCover: get the URL of the cover image
videos_df['coverImage'] = videos_df.apply(lambda row: parse_video(row['video']), axis=1)

videos_df


,createTime,desc,id,music,video,textExtra,warnInfo,desc_en,textExtra_en,collectCount,commentCount,diggCount,playCount,shareCount,coverImage
0,2023-09-13 18:57:03,"La mia intervista di questa sera a ""Cinque Min...",7278386520275373056,"{'authorName': 'Giorgia Meloni', 'coverLarge':...","{'bitrate': 1178262, 'bitrateInfo': [{'Bitrate...",None,None,"My interview this evening on ""Cinque Minuti"", ...",None,840,2669,14800,435800,438,https://p16-sign-useast2a.tiktokcdn.com/tos-us...
1,2023-09-12 16:05:41,Stiamo dando alla Nazione una strategia che no...,7277971263958666240,"{'authorName': 'Giorgia Meloni', 'coverLarge':...","{'bitrate': 1021716, 'bitrateInfo': [{'Bitrate...",None,None,We are giving the Nation a strategy that it ha...,None,1050,6426,21200,622700,598,https://p16-sign-useast2a.tiktokcdn.com/tos-us...
2,2023-09-11 09:02:24,"Grazie India, complimenti per il successo del ...",7277491085071387648,"{'authorName': 'Giorgia Meloni', 'coverLarge':...","{'bitrate': 1374581, 'bitrateInfo': [{'Bitrate...",g20,None,"Thank you India, congratulations on the succes...",g20,313,468,7158,137500,224,https://p16-sign-useast2a.tiktokcdn.com/tos-us...
3,2023-09-03 07:28:10,A 41 anni dal brutale attentato mafioso che ha...,7274498125660703744,"{'authorName': 'Giorgia Meloni', 'coverLarge':...","{'bitrate': 627490, 'bitrateInfo': [{'Bitrate'...",None,None,41 years after the brutal mafia attack that ca...,None,277,274,4874,141400,204,https://p16-sign-useast2a.tiktokcdn.com/tos-us...
4,2023-08-12 06:05:11,Salario minimo: punto stampa dopo l’incontro c...,7266312532665584640,"{'authorName': 'Giorgia Meloni', 'coverLarge':...","{'bitrate': 1077066, 'bitrateInfo': [{'Bitrate...",None,None,Minimum wage: press point after the meeting wi...,None,2847,7144,52600,1800000,1388,https://p16-sign-useast2a.tiktokcdn.com/tos-us...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2022-02-20 17:31:35,Il risultato del #greenpass è stato solo quell...,7066845965105318912,"{'authorName': 'Giorgia Meloni', 'coverLarge':...","{'bitrate': 998202, 'bitrateInfo': [{'Bitrate'...",greenpass,None,The result of the #greenpass was only to manag...,greenpass,2460,2020,54800,977100,4925,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...
220,2022-02-18 17:00:34,Noi vogliamo difendere l’Italia 🇮🇹,7066095800291576832,"{'authorName': 'Giorgia Meloni', 'coverLarge':...","{'bitrate': 710139, 'bitrateInfo': [{'Bitrate'...",None,None,We want to defend Italy 🇮🇹,None,30,41,2052,37700,37,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...
221,2022-02-11 14:53:48,Sia chiaro a tutti: noi il governo con il PD e...,7063465536109235200,"{'authorName': 'Giorgia Meloni', 'coverLarge':...","{'bitrate': 401098, 'bitrateInfo': [{'Bitrate'...",None,None,Let it be clear to everyone: we will never for...,None,22,40,1482,33000,21,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...
222,2022-02-10 19:07:13,Cittadini e imprese schiacciati dalla crisi e ...,7063159758185827328,"{'authorName': 'Giorgia Meloni', 'coverLarge':...","{'bitrate': 326414, 'bitrateInfo': [{'Bitrate'...",None,None,Citizens and businesses crushed by the crisis ...,None,28,202,2240,47300,126,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...


Write out the results to a CSV file. 

In [14]:
videos_df.to_json('../data/processed/videos-translated.json')
videos_df.to_csv('../data/processed/videos-translated.csv', sep=',')

videos_df[['createTime', 'textExtra_en','desc_en']].to_csv('../data/processed/hashtags-and-captions-translated.csv', sep=',')

## Download Resources

In [82]:
# def download_file_and_save(url: str, filepath: Path):
#     """Download a file from a specified URL and write its contents to a file"""

#     r = _get(url=url)
#     if r.status_code == 403:
#         return
#     ext = r.headers["Content-Type"].split("/")[-1]
#     path_with_ext = filepath.with_suffix(f".{ext}")
#     with open(path_with_ext, "wb") as f:
#         f.write(r.content)
#         logger.debug(f"Saved file to: {path_with_ext}")

videos_df['coverImage']

# get the video Duration


0      https://p16-sign-useast2a.tiktokcdn.com/tos-us...
1      https://p16-sign-useast2a.tiktokcdn.com/tos-us...
2      https://p16-sign-useast2a.tiktokcdn.com/tos-us...
3      https://p16-sign-useast2a.tiktokcdn.com/tos-us...
4      https://p16-sign-useast2a.tiktokcdn.com/tos-us...
                             ...                        
219    https://p16-sign-va.tiktokcdn.com/tos-maliva-p...
220    https://p16-sign-va.tiktokcdn.com/tos-maliva-p...
221    https://p16-sign-va.tiktokcdn.com/tos-maliva-p...
222    https://p16-sign-va.tiktokcdn.com/tos-maliva-p...
223    https://p16-sign-va.tiktokcdn.com/tos-maliva-p...
Name: coverImage, Length: 224, dtype: object

## Download Videos

In [13]:
import logging
import os
import json
from pathlib import Path
from urllib.error import HTTPError
import yt_dlp
from yt_dlp.utils import ExtractorError, DownloadError
from typing import List, Dict, Optional

logger = logging.getLogger(__name__)

def download_videos(url_to_download=None, output_dir=None):
    if len(url_to_download) > 0:
        print(f"Downloading media {url_to_download}")
        logger.info(f"Downloading media for hashtag {url_to_download}")

    ydl_opts = {
        "outtmpl": os.path.join(output_dir, "%(id)s.%(ext)s"),
        "ignore_errors": True,
        "quiet": logger.getEffectiveLevel() > logging.DEBUG,
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url_to_download])
        except (HTTPError, TypeError, ExtractorError, DownloadError) as e:
            logger.warning(
                f"Encountered error {e} when attempting to download url: {url_to_download}"
            )

def get_video_list(videos_dir):
    already_downloaded_ids = set(
        file.split(".")[0].split("_")[0] for file in os.listdir(videos_dir)
    )
    video_df = pd.json_normalize(videos_df['video'])
    return video_df[~video_df['id'].isin(already_downloaded_ids)]
 
new_videos = get_video_list(videos_dir='../data/processed/videos')
new_videos
for video_id in new_videos['id']:
    print(video_id)
    user_id = '7057902765381534725'
    url = f"https://www.tiktok.com/@{user_id}/video/{video_id}"
    print(url)
    download_videos(url_to_download=url, output_dir='../data/processed/videos')

## Update the Videos CSV/JSON with video duration

In [115]:
def get_video_duration(filename):

    video = cv2.VideoCapture(filename)

    if video is None or not video.isOpened():
       print('Warning: unable to open video source: ', filename)
       return '###### VIDEO NOT FOUND ###########'

    duration = video.get(cv2.CAP_PROP_POS_MSEC)
    fps = video.get(cv2.CAP_PROP_FPS)      
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps

    minutes = int(duration/60)
    seconds = duration%60
    duration_formatted = str(minutes) + ':' + str(round(seconds))

    video.release()
    return duration_formatted

videos_translated_df = pd.read_json(processed_dir + 'videos-translated.json', convert_dates=['createTime'])

def add_duration(file_id):
    file_name = processed_dir + 'videos/' + str(file_id) + '.mp4'
    duration = get_video_duration(file_name)
    
    print(duration)
    return duration

videos_translated_df['duration'] = videos_translated_df.apply(lambda row: add_duration(row['id']), axis=1)

videos_translated_df.to_json(processed_dir + 'videos-translated-with-duration.json')



###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOUND ###########
###### VIDEO NOT FOU

OpenCV: Couldn't read video stream from file "../data/processed/videos/7278386520275373000.mp4"
[ERROR:0@3832.463] global cap.cpp:166 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


OpenCV: Couldn't read video stream from file "../data/processed/videos/7277971263958666000.mp4"
[ERROR:0@3832.463] global cap.cpp:166 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


OpenCV: Couldn't read video stream from file "../data/processed/videos/7277491085071388000.mp4"
[ERROR:0@3832.464] global cap.cpp:166 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /

## Convert any Video Files to audio 

In [15]:
import moviepy
import moviepy.editor
import os
import glob

videos_dir = '../data/processed/videos/'
audio_dir = '../data/processed/audio'

if not os.path.exists(audio_dir):
    os.mkdir(audio_dir)

file_name_list = glob.glob(os.path.join(videos_dir, "*.mp4"))
print(len(file_name_list))

for file_name in file_name_list:
    # print(file_name)
    video = moviepy.editor.VideoFileClip(file_name)
    audio = video.audio
    audio_file_name = file_name.replace('videos','audio').replace('.mp4', '.mp3')
    audio.write_audiofile(audio_file_name)

224
MoviePy - Writing audio in ../data/processed/audio/7144425341279341830.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/processed/audio/7187471251794005254.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/processed/audio/7145754357013777670.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/processed/audio/7158424452286401797.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/processed/audio/7277971263958666529.mp3


...

MoviePy - Done.
MoviePy - Writing audio in ../data/processed/audio/7238646546152082714.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/processed/audio/7092396552064453893.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/processed/audio/7266312532665584928.mp3


MoviePy - Done.
MoviePy - Writing audio in ../data/processed/audio/7235728376848272666.mp3


MoviePy - Done.


## Extract Transcribed Text

In [79]:
transcription_dir_location = '../data/processed/transcription/'

def extract_translated_text(file_name: str):
    if file_name == '.DS_Store':
        return
    
    print(f'file_name: {file_name}')
    transcriptions_df = pd.read_json(transcription_dir_location + 'original/' + file_name)
    transcripts_df = transcriptions_df['results']['transcripts'] 
    if len(transcripts_df) > 1:
        print('error')
        raise

    transcript_df = pd.DataFrame.from_dict(transcripts_df[0], orient='index')
    print(transcript_df)
    transcript_df.to_json(f'{transcription_dir_location}/{file_name}')    

for file_name in os.listdir(transcription_dir_location + 'original'):
    extract_translated_text(file_name)



file_name: TranscribeTikTokAudio7216786331005930778.json
                                                            0
transcript  Un altro capitolo importante del nostro lavoro...
file_name: TranscribeTikTokAudio7177010908625980678.json
                                                            0
transcript  ricorrono quest'anno i dieci anni dalla fondaz...
file_name: TranscribeTikTokAudio7178163802578455813.json
           0
transcript  
file_name: TranscribeTikTokAudio7194909784926063877.json
                                                            0
transcript  a cento giorni dall'insediamento del nuovo gov...
file_name: TranscribeTikTokAudio7092079134247046405.json
                                         0
transcript  Sì, ma io la Presidente. Ciao.
file_name: TranscribeTikTokAudio7146670520505847045.json
                                                            0
transcript  sarebbe stato facile in questa campagna eletto...
file_name: TranscribeTikTokAudio716894605970676455